# Tweets abnehmen und Sentiment analyse anwenden

Die Tweets aus dem Producer werden eingelesen und als PArquet abgelegt.
Darauf wird via *textblob* eine Sentiment analyse gemacht und jedem Tweet ein Rating zugewiesen.

In [0]:
 !pip install textblob

In [0]:
# Libraries einlesen

from pyspark import SparkContext, SparkConf
from pyspark.streaming import StreamingContext
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql import functions as F
from textblob import TextBlob


In [0]:
# nur zum testen: stream sichtbar machen

# Create a local/private StreamingContext and SparkContext.
ssc = StreamingContext(sc, 5)   # window size = 5
stream = ssc.socketTextStream("localhost", 9997)
stream.pprint()

# Start the computation with timeout function
 try:
  ssc.start()                             
  ssc.awaitTerminationOrTimeout(20)  # Ausgabe im consumer erst nach timeout möglich (sekunden)
 finally:
  ssc.stop(False)


In [0]:
# Tokenize and preprocess text

def preprocessing(lines):
    words = lines.select(explode(split(lines.value, "t_end")).alias("word"))
    words = words.na.replace('', None)
    words = words.na.drop()
    words = words.withColumn('word', F.regexp_replace('word', r'http\S+', ''))
    words = words.withColumn('word', F.regexp_replace('word', '@\w+', ''))
    words = words.withColumn('word', F.regexp_replace('word', '#', ''))
    words = words.withColumn('word', F.regexp_replace('word', 'RT', ''))
    words = words.withColumn('word', F.regexp_replace('word', ':', ''))
    return words


In [0]:
# Apply text classification via Textblob

def polarity_detection(text):
    return TextBlob(text).sentiment.polarity
  
def subjectivity_detection(text):
    return TextBlob(text).sentiment.subjectivity

def text_classification(words):
    # polarity detection
    polarity_detection_udf = udf(polarity_detection, StringType())
    words = words.withColumn("polarity", polarity_detection_udf("word"))
    # subjectivity detection
    subjectivity_detection_udf = udf(subjectivity_detection, StringType())
    words = words.withColumn("subjectivity", subjectivity_detection_udf("word"))
    return words
  

In [0]:
# Load in the Tweets and apply textblob sentiment

if __name__ == "__main__":
    # create Spark session
    spark = SparkSession.builder.appName("TwitterSentimentAnalysis").getOrCreate()

    # read the tweet data from socket
    lines = spark.readStream.format("socket").option("host", 'localhost').option("port", 9997).load()
    # Preprocess the data
    words = preprocessing(lines)
    # text classification to define polarity and subjectivity
    words = text_classification(words)

    #write tweets into parquet files every 60 sec
    words = words.repartition(1)
    query = words.writeStream.queryName("all_tweets")\
        .outputMode("append").format("parquet")\
        .option("path", "dbfs:/FileStore/bd_project")\
        .option("checkpointLocation", "./check")\
        .trigger(processingTime='60 seconds').start()
    query.awaitTermination()
   